In [2]:
import matplotlib.pyplot as plt
import numpy as np
import yaml
from pathlib import Path
from regions import CircleSkyRegion
from astropy import units as u
from astropy.coordinates import SkyCoord
from gammapy.analysis import Analysis, AnalysisConfig
from gammapy.utils.scripts import read_yaml

import gammapy ; print(gammapy.__version__)

0.15.dev647+gef80dc1eb


In [38]:
#!/usr/bin/env python
"""Run Gammapy validation: CTA 1DC"""
import logging
from pathlib import Path

import yaml
import astropy.units as u
from astropy.coordinates import Angle
from gammapy.analysis import Analysis, AnalysisConfig

log = logging.getLogger(__name__)


def get_config(target):
    config = yaml.safe_load(open("targets.yaml"))
    return config[target]


def cli():
    targets = "all"
    if targets == "all":
        targets = ["cas_a", "hess_j1702"]
    else:
        targets = targets.split(",")

    for target in targets:
        analysis_3d(target)


def analysis_3d(target):
    log.info(f"analysis_3d: {target}")
    analysis_3d_data_reduction(target)
    analysis_3d_modeling(target)
    analysis_3d_summary(target)


def analysis_3d_data_reduction(target):
    log.info(f"analysis_3d_data_reduction: {target}")

    opts = get_config(target)
    # TODO: remove this once Gammapy config is more uniform
    opts["emin_tev"] = u.Quantity(opts["emin"]).to_value("TeV")
    opts["emax_tev"] = u.Quantity(opts["emax"]).to_value("TeV")
    opts["lon_deg"] = Angle(opts["lon"]).deg
    opts["lat_deg"] = Angle(opts["lat"]).deg

    txt = Path("config_template.yaml").read_text()
    txt = txt.format_map(opts)
    config = yaml.safe_load(txt)
    config = AnalysisConfig(config)

    analysis = Analysis(config)
    analysis.get_observations()

    log.info("Running data reduction")
    analysis.get_datasets()

    # TODO: write datasets and separate fitting to next function
    # Not implemented in Gammapy yet, coming very soon.
    log.info("Running fit ...")
    analysis.set_model(filename=f"{target}/model_3d.yaml")
    logging.info(analysis.model)
    analysis.run_fit()
    logging.info(analysis.fit_result.parameters.to_table())
    path = f"{target}/{target}_3d_bestfit.rst"
    log.info(f"Writing {path}")
    analysis.fit_result.parameters.to_table().write(path, format="ascii.rst", overwrite=True)

    analysis.get_flux_points(source=f"{target}")
    path = f"{target}/{target}_3d_fluxpoints.fits"
    log.info(f"Writing {path}")
    analysis.flux_points.write(path, overwrite=True)

    return analysis

def analysis_3d_modeling(target):
    log.info(f"analysis_3d_modeling: {target}")


def analysis_3d_summary(target):
    log.info(f"analysis_3d_summary: {target}")
    # TODO: make plots
    # TODO: summarise results to `results.md`? Necessary?

#if __name__ == "__main__":
#    logging.basicConfig(level=logging.INFO)
#    cli()



In [39]:
analysis = analysis_3d('cas_a')

INFO:__main__:analysis_3d: cas_a
INFO:__main__:analysis_3d_data_reduction: cas_a
INFO:gammapy.analysis.core:Setting logging config: {'level': 'INFO'}
INFO:gammapy.analysis.core:Fetching observations.
INFO:gammapy.analysis.core:15 observations were selected.
INFO:__main__:Running data reduction
INFO:gammapy.analysis.core:Creating geometry.
INFO:gammapy.analysis.core:Creating datasets.
INFO:gammapy.analysis.core:Processing observation 112301
INFO:gammapy.analysis.core:Processing observation 112302
INFO:gammapy.analysis.core:Processing observation 112303
INFO:gammapy.analysis.core:Processing observation 112304
INFO:gammapy.analysis.core:Processing observation 112305
INFO:gammapy.analysis.core:Processing observation 112306
INFO:gammapy.analysis.core:Processing observation 112307
INFO:gammapy.analysis.core:Processing observation 112308
INFO:gammapy.analysis.core:Processing observation 112309
INFO:gammapy.analysis.core:Processing observation 112310
INFO:gammapy.analysis.core:Processing obser

In [77]:
from astropy.table import Table

target='cas_a'


path = f"{target}/{target}_3d_bestfit.rst"

tab=Table.read(path, format='ascii')
tab.add_index('name')
pars=['lon_0','lat_0','index','amplitude'] #should be read from yaml model file

dt = 'U30'
comp_tab = Table( names=('Param', 'DC1 Ref', 'gammapy 3d'), dtype=[dt,dt,dt] )

for par in pars:
    value =tab.loc[par]['value']
    name =tab.loc[par]['name']
    error =tab.loc[par]['error']
    comp_tab.add_row([name, '2',f"{value}±{error}"], )


path = f"{target}/README.md"

comp_tab.write()

#tab.write('toto.rst', format='ascii.rst')

IndexError: tuple index out of range

In [128]:
gauss_model = """
components:
- name: hess_j1702
  type: SkyModel
  spatial:
    type: GaussianSpatialModel
    frame: galactic
    parameters:
    - name: lon_0
      value: 344.3
      unit: deg
    - name: lat_0 
      value: -0.1847
      unit: deg
    - name: sigma
      value: 0.3
      unit: deg
    - name: e
      value: 0
      unit: ''
      frozen: True
    - name: phi
      value: 0
      unit: deg
      frozen: True 
  spectral:
    type: PowerLawSpectralModel
    parameters:
    - name: amplitude
      value: 9.0e-12
      unit: cm-2 s-1 TeV-1
    - name: index
      value: 2.0
      unit: ''
    - name: reference
      value: 1.0
      unit: TeV
      frozen: true

"""
analysis.set_model(model=gauss_model)

INFO:gammapy.analysis.core:Reading model.


ValueError: cannot convert float NaN to integer

In [ ]:
run_3d('J1702')
